In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [17]:
# fix random seed for reproducibility
tf.random.set_seed(7)

In [18]:
# load the dataset
dataframe = pd.read_csv('../data/processed/cangas_IU.csv', usecols=[2], engine='python')
dataframe


,abierto/cerrado
0,0
1,0
2,0
3,0
4,0
...,...
3282,0
3283,0
3284,0
3285,0


In [19]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [20]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

2202 1085


In [21]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [22]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [23]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [24]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


C:\Users\rmarc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2200/2200 - 2s - 929us/step - loss: nan
Epoch 2/100
2200/2200 - 1s - 575us/step - loss: nan
Epoch 3/100
2200/2200 - 1s - 559us/step - loss: nan
Epoch 4/100
2200/2200 - 1s - 571us/step - loss: nan
Epoch 5/100
2200/2200 - 1s - 566us/step - loss: nan
Epoch 6/100
2200/2200 - 1s - 560us/step - loss: nan
Epoch 7/100
2200/2200 - 1s - 578us/step - loss: nan
Epoch 8/100
2200/2200 - 1s - 577us/step - loss: nan
Epoch 9/100
2200/2200 - 1s - 583us/step - loss: nan
Epoch 10/100
2200/2200 - 1s - 568us/step - loss: nan
Epoch 11/100
2200/2200 - 1s - 572us/step - loss: nan
Epoch 12/100
2200/2200 - 1s - 573us/step - loss: nan
Epoch 13/100
2200/2200 - 1s - 564us/step - loss: nan
Epoch 14/100
2200/2200 - 1s - 547us/step - loss: nan
Epoch 15/100
2200/2200 - 1s - 543us/step - loss: nan
Epoch 16/100
2200/2200 - 1s - 549us/step - loss: nan
Epoch 17/100
2200/2200 - 1s - 554us/step - loss: nan
Epoch 18/100
2200/2200 - 1s - 549us/step - loss: nan
Epoch 19/100
2200/2200 - 1s - 543us/step - loss: nan
Epoch 20/100
2

In [26]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step


ValueError: Found array with dim 3. None expected <= 2.